In [1]:
# %tb

# Import Cell Profiler Dependencies
import cellprofiler
import cellprofiler_core.image
import cellprofiler_core.measurement
import cellprofiler_core.object
import cellprofiler_core.pipeline
import cellprofiler_core.preferences
import cellprofiler_core.workspace
from cellprofiler_core.utilities.java import start_java, stop_java




# # Inject Image module used to inject OMERO image planes into Cell Profiler Pipeline
# from cellprofiler_core.modules.injectimage import InjectImage

# Import OMERO Python BlitzGateway
# import omero
# from omero.gateway import BlitzGateway

# Import Numpy
import numpy as np

import random
import string

# Import Python System Packages
import os
import tempfile
import warnings


import io
import os
import pandas as pd
import plotly.express as px
import plotly.io as pio
import skimage.io

pio.renderers.default = "png"

# mura model imports
import requests
import math
import matplotlib.pyplot as plt
import shutil
from getpass import getpass
from PIL import Image, UnidentifiedImageError
from requests.exceptions import HTTPError
from io import BytesIO
from pathlib import Path
import torch
import pytorch_lightning as pl
# from huggingface_hub import HfApi, HfFolder, Repository, notebook_login
from torch.utils.data import DataLoader
from torchmetrics import Accuracy
from torchvision.datasets import ImageFolder
# from transformers import ViTFeatureExtractor, ViTForImageClassification
from transformers import ViTImageProcessor, ViTForImageClassification

set output directory

In [2]:
# Helper functions

profile_formatter = (
    "s3://cellpainting-gallery/cpg0016-jump/"
    "{Metadata_Source}/workspace/profiles/"
    "{Metadata_Batch}/{Metadata_Plate}/{Metadata_Plate}.parquet"
)

loaddata_formatter = (
    "s3://cellpainting-gallery/cpg0016-jump/"
    "{Metadata_Source}/workspace/load_data_csv/"
    "{Metadata_Batch}/{Metadata_Plate}/load_data_with_illum.parquet"
)

pull images

In [13]:
temp_image_path = "/Users/abestroka/Argonne/git_repos/JUMP_vision_model/image_temp"

Loop through linked list, pull each image, and segment with cell profiler pipeline, then train model

In [14]:


! cellprofiler -c -r -p /Users/abestroka/Argonne/git_repos/JUMP_vision_model/my_project_421.cppipe -i /Users/abestroka/Argonne/git_repos/JUMP_vision_model/image_temp -o /Users/abestroka/Argonne/git_repos/JUMP_vision_model/segmented_image_temp/{i}
cells_path = "/Users/abestroka/Argonne/git_repos/JUMP_vision_model/segmented_image_temp"

#place images into correct target folder
#check if target folder exists, if not, create one
dst_dir = os.path.join(cells_path, target)
src_dir = os.path.join(cells_path, str(i))
exists = os.path.exists(dst_dir)
if exists == False:
#         new_dir = os.path.join(cells_path, target)
    os.mkdir(dst_dir) 

#transfer images

dst_images = os.listdir(dst_dir)
#     print(dst_images)
src_images = os.listdir(src_dir)
#     print(src_images)

for image in src_images:
    curr_path = os.path.join(src_dir, image)
#         if image in dst_images:
    # rename so no overwrite
    new_name = ''.join(random.choices(string.ascii_lowercase + string.digits, k=20))
    new_name = new_name + ".png"
    new_path = os.path.join(src_dir, new_name)
    os.rename(curr_path, new_path)

    #transfer image
    shutil.copy(new_path, dst_dir)

#     dst_images = os.listdir(dst_dir)
#     print(dst_images)
#     src_images = os.listdir(src_dir)
#     print(src_images)

# delete temp directory
shutil.rmtree(src_dir)


# clear folder with images

if i == 100:
    break
else:
    i+=1

0
TARGET SRVFFFJZQVENJC-UHFFFAOYSA-N
download: s3://cellpainting-gallery/cpg0016-jump/source_11/images/Batch3/images/EC000133__2021-09-24T18_01_09-Measurement1/Images/r01c01f01p01-ch1sk1fk1fl1.tiff to image_temp/r01c01f01p01-ch1sk1fk1fl1.tiff
download: s3://cellpainting-gallery/cpg0016-jump/source_11/images/Batch3/images/EC000133__2021-09-24T18_01_09-Measurement1/Images/r01c01f01p01-ch2sk1fk1fl1.tiff to image_temp/r01c01f01p01-ch2sk1fk1fl1.tiff
download: s3://cellpainting-gallery/cpg0016-jump/source_11/images/Batch3/images/EC000133__2021-09-24T18_01_09-Measurement1/Images/r01c01f01p01-ch4sk1fk1fl1.tiff to image_temp/r01c01f01p01-ch4sk1fk1fl1.tiff
download: s3://cellpainting-gallery/cpg0016-jump/source_11/images/Batch3/images/EC000133__2021-09-24T18_01_09-Measurement1/Images/r01c01f01p01-ch3sk1fk1fl1.tiff to image_temp/r01c01f01p01-ch3sk1fk1fl1.tiff
download: s3://cellpainting-gallery/cpg0016-jump/source_11/images/Batch3/images/EC000133__2021-09-24T18_01_09-Measurement1/Images/r01c01f01

download: s3://cellpainting-gallery/cpg0016-jump/source_11/images/Batch3/images/EC000133__2021-09-24T18_01_09-Measurement1/Images/r01c01f03p01-ch5sk1fk1fl1.tiff to image_temp/r01c01f03p01-ch5sk1fk1fl1.tiff
download: s3://cellpainting-gallery/cpg0016-jump/source_11/images/Batch3/illum/EC000133/EC000133_IllumDNA.npy to image_temp/EC000133_IllumDNA.npy
download: s3://cellpainting-gallery/cpg0016-jump/source_11/images/Batch3/illum/EC000133/EC000133_IllumER.npy to image_temp/EC000133_IllumER.npy
download: s3://cellpainting-gallery/cpg0016-jump/source_11/images/Batch3/illum/EC000133/EC000133_IllumRNA.npy to image_temp/EC000133_IllumRNA.npy
download: s3://cellpainting-gallery/cpg0016-jump/source_11/images/Batch3/illum/EC000133/EC000133_IllumAGP.npy to image_temp/EC000133_IllumAGP.npy
download: s3://cellpainting-gallery/cpg0016-jump/source_11/images/Batch3/illum/EC000133/EC000133_IllumMito.npy to image_temp/EC000133_IllumMito.npy
Times reported are CPU and Wall-clock times for each module
Thu 

Times reported are CPU and Wall-clock times for each module
Thu Jan 11 13:53:35 2024: Image # 1, module Images # 1: CPU_time = 0.00 secs, Wall_time = 0.00 secs
Thu Jan 11 13:53:35 2024: Image # 1, module Metadata # 2: CPU_time = 0.00 secs, Wall_time = 0.00 secs
Thu Jan 11 13:53:35 2024: Image # 1, module NamesAndTypes # 3: CPU_time = 4.02 secs, Wall_time = 1.66 secs
Thu Jan 11 13:53:36 2024: Image # 1, module Groups # 4: CPU_time = 0.00 secs, Wall_time = 0.00 secs
Thu Jan 11 13:53:36 2024: Image # 1, module CorrectIlluminationApply # 5: CPU_time = 0.25 secs, Wall_time = 0.26 secs
Thu Jan 11 13:53:37 2024: Image # 1, module CorrectIlluminationCalculate # 6: CPU_time = 0.40 secs, Wall_time = 0.40 secs
Thu Jan 11 13:53:37 2024: Image # 1, module CorrectIlluminationApply # 7: CPU_time = 0.00 secs, Wall_time = 0.00 secs
Thu Jan 11 13:53:37 2024: Image # 1, module IdentifyPrimaryObjects # 8: CPU_time = 0.56 secs, Wall_time = 0.55 secs
Thu Jan 11 13:53:38 2024: Image # 1, module IdentifySecon

Thu Jan 11 13:55:28 2024: Image # 1, module IdentifySecondaryObjects # 9: CPU_time = 0.71 secs, Wall_time = 0.72 secs
Thu Jan 11 13:55:29 2024: Image # 1, module IdentifyTertiaryObjects # 10: CPU_time = 0.20 secs, Wall_time = 0.21 secs
Thu Jan 11 13:55:29 2024: Image # 1, module RescaleIntensity # 11: CPU_time = 0.11 secs, Wall_time = 0.11 secs
Thu Jan 11 13:55:29 2024: Image # 1, module RescaleIntensity # 12: CPU_time = 0.11 secs, Wall_time = 0.11 secs
Thu Jan 11 13:55:29 2024: Image # 1, module GrayToColor # 13: CPU_time = 0.06 secs, Wall_time = 0.06 secs
Thu Jan 11 13:55:29 2024: Image # 1, module ImageMath # 14: CPU_time = 0.00 secs, Wall_time = 0.00 secs
Thu Jan 11 13:55:29 2024: Image # 1, module OverlayOutlines # 15: CPU_time = 0.06 secs, Wall_time = 0.06 secs
Thu Jan 11 13:55:29 2024: Image # 1, module SaveCroppedObjects # 17: CPU_time = 3.42 secs, Wall_time = 3.44 secs
True
7
TARGET SRVFFFJZQVENJC-UHFFFAOYSA-N
download: s3://cellpainting-gallery/cpg0016-jump/source_11/images/B

download: s3://cellpainting-gallery/cpg0016-jump/source_11/images/Batch3/images/EC000133__2021-09-24T18_01_09-Measurement1/Images/r01c02f01p01-ch1sk1fk1fl1.tiff to image_temp/r01c02f01p01-ch1sk1fk1fl1.tiff
download: s3://cellpainting-gallery/cpg0016-jump/source_11/images/Batch3/images/EC000133__2021-09-24T18_01_09-Measurement1/Images/r01c02f01p01-ch2sk1fk1fl1.tiff to image_temp/r01c02f01p01-ch2sk1fk1fl1.tiff
download: s3://cellpainting-gallery/cpg0016-jump/source_11/images/Batch3/images/EC000133__2021-09-24T18_01_09-Measurement1/Images/r01c02f01p01-ch4sk1fk1fl1.tiff to image_temp/r01c02f01p01-ch4sk1fk1fl1.tiff
download: s3://cellpainting-gallery/cpg0016-jump/source_11/images/Batch3/images/EC000133__2021-09-24T18_01_09-Measurement1/Images/r01c02f01p01-ch3sk1fk1fl1.tiff to image_temp/r01c02f01p01-ch3sk1fk1fl1.tiff
download: s3://cellpainting-gallery/cpg0016-jump/source_11/images/Batch3/images/EC000133__2021-09-24T18_01_09-Measurement1/Images/r01c02f01p01-ch5sk1fk1fl1.tiff to image_temp/r

...

Thu Jan 11 15:13:53 2024: Image # 1, module CorrectIlluminationCalculate # 6: CPU_time = 0.41 secs, Wall_time = 0.40 secs
Thu Jan 11 15:13:54 2024: Image # 1, module CorrectIlluminationApply # 7: CPU_time = 0.00 secs, Wall_time = 0.00 secs
Thu Jan 11 15:13:54 2024: Image # 1, module IdentifyPrimaryObjects # 8: CPU_time = 0.58 secs, Wall_time = 0.58 secs
Thu Jan 11 15:13:54 2024: Image # 1, module IdentifySecondaryObjects # 9: CPU_time = 0.62 secs, Wall_time = 0.62 secs
Thu Jan 11 15:13:55 2024: Image # 1, module IdentifyTertiaryObjects # 10: CPU_time = 0.20 secs, Wall_time = 0.20 secs
Thu Jan 11 15:13:55 2024: Image # 1, module RescaleIntensity # 11: CPU_time = 0.11 secs, Wall_time = 0.11 secs
Thu Jan 11 15:13:55 2024: Image # 1, module RescaleIntensity # 12: CPU_time = 0.10 secs, Wall_time = 0.11 secs
Thu Jan 11 15:13:55 2024: Image # 1, module GrayToColor # 13: CPU_time = 0.06 secs, Wall_time = 0.06 secs
Thu Jan 11 15:13:55 2024: Image # 1, module ImageMath # 14: CPU_time = 0.00 secs

Thu Jan 11 15:15:51 2024: Image # 1, module OverlayOutlines # 15: CPU_time = 0.05 secs, Wall_time = 0.05 secs
Thu Jan 11 15:15:51 2024: Image # 1, module SaveCroppedObjects # 17: CPU_time = 3.04 secs, Wall_time = 3.06 secs
True
93
TARGET OZXFTSNPYNIBCW-UHFFFAOYSA-N
download: s3://cellpainting-gallery/cpg0016-jump/source_11/images/Batch3/images/EC000133__2021-09-24T18_01_09-Measurement1/Images/r01c11f04p01-ch1sk1fk1fl1.tiff to image_temp/r01c11f04p01-ch1sk1fk1fl1.tiff
download: s3://cellpainting-gallery/cpg0016-jump/source_11/images/Batch3/images/EC000133__2021-09-24T18_01_09-Measurement1/Images/r01c11f04p01-ch2sk1fk1fl1.tiff to image_temp/r01c11f04p01-ch2sk1fk1fl1.tiff
download: s3://cellpainting-gallery/cpg0016-jump/source_11/images/Batch3/images/EC000133__2021-09-24T18_01_09-Measurement1/Images/r01c11f04p01-ch4sk1fk1fl1.tiff to image_temp/r01c11f04p01-ch4sk1fk1fl1.tiff
download: s3://cellpainting-gallery/cpg0016-jump/source_11/images/Batch3/images/EC000133__2021-09-24T18_01_09-Measur

download: s3://cellpainting-gallery/cpg0016-jump/source_11/images/Batch3/images/EC000133__2021-09-24T18_01_09-Measurement1/Images/r01c11f06p01-ch3sk1fk1fl1.tiff to image_temp/r01c11f06p01-ch3sk1fk1fl1.tiff
download: s3://cellpainting-gallery/cpg0016-jump/source_11/images/Batch3/images/EC000133__2021-09-24T18_01_09-Measurement1/Images/r01c11f06p01-ch5sk1fk1fl1.tiff to image_temp/r01c11f06p01-ch5sk1fk1fl1.tiff
download: s3://cellpainting-gallery/cpg0016-jump/source_11/images/Batch3/illum/EC000133/EC000133_IllumDNA.npy to image_temp/EC000133_IllumDNA.npy
download: s3://cellpainting-gallery/cpg0016-jump/source_11/images/Batch3/illum/EC000133/EC000133_IllumER.npy to image_temp/EC000133_IllumER.npy
download: s3://cellpainting-gallery/cpg0016-jump/source_11/images/Batch3/illum/EC000133/EC000133_IllumRNA.npy to image_temp/EC000133_IllumRNA.npy
download: s3://cellpainting-gallery/cpg0016-jump/source_11/images/Batch3/illum/EC000133/EC000133_IllumAGP.npy to image_temp/EC000133_IllumAGP.npy
downlo

download: s3://cellpainting-gallery/cpg0016-jump/source_11/images/Batch3/illum/EC000133/EC000133_IllumMito.npy to image_temp/EC000133_IllumMito.npy
Times reported are CPU and Wall-clock times for each module
Thu Jan 11 15:20:26 2024: Image # 1, module Images # 1: CPU_time = 0.00 secs, Wall_time = 0.00 secs
Thu Jan 11 15:20:26 2024: Image # 1, module Metadata # 2: CPU_time = 0.00 secs, Wall_time = 0.00 secs
Thu Jan 11 15:20:26 2024: Image # 1, module NamesAndTypes # 3: CPU_time = 3.15 secs, Wall_time = 1.85 secs
Thu Jan 11 15:20:28 2024: Image # 1, module Groups # 4: CPU_time = 0.00 secs, Wall_time = 0.00 secs
Thu Jan 11 15:20:28 2024: Image # 1, module CorrectIlluminationApply # 5: CPU_time = 0.37 secs, Wall_time = 0.27 secs
Thu Jan 11 15:20:28 2024: Image # 1, module CorrectIlluminationCalculate # 6: CPU_time = 0.41 secs, Wall_time = 0.40 secs
Thu Jan 11 15:20:29 2024: Image # 1, module CorrectIlluminationApply # 7: CPU_time = 0.00 secs, Wall_time = 0.00 secs
Thu Jan 11 15:20:29 2024:

Thu Jan 11 15:22:20 2024: Image # 1, module IdentifyPrimaryObjects # 8: CPU_time = 0.56 secs, Wall_time = 0.56 secs
Thu Jan 11 15:22:21 2024: Image # 1, module IdentifySecondaryObjects # 9: CPU_time = 0.57 secs, Wall_time = 0.57 secs
Thu Jan 11 15:22:21 2024: Image # 1, module IdentifyTertiaryObjects # 10: CPU_time = 0.18 secs, Wall_time = 0.19 secs
Thu Jan 11 15:22:22 2024: Image # 1, module RescaleIntensity # 11: CPU_time = 0.10 secs, Wall_time = 0.11 secs
Thu Jan 11 15:22:22 2024: Image # 1, module RescaleIntensity # 12: CPU_time = 0.10 secs, Wall_time = 0.11 secs
Thu Jan 11 15:22:22 2024: Image # 1, module GrayToColor # 13: CPU_time = 0.04 secs, Wall_time = 0.06 secs
Thu Jan 11 15:22:22 2024: Image # 1, module ImageMath # 14: CPU_time = 0.00 secs, Wall_time = 0.00 secs
Thu Jan 11 15:22:22 2024: Image # 1, module OverlayOutlines # 15: CPU_time = 0.05 secs, Wall_time = 0.05 secs
Thu Jan 11 15:22:22 2024: Image # 1, module SaveCroppedObjects # 17: CPU_time = 2.14 secs, Wall_time = 2.1

In [15]:
#     #init dataset, split into training and validation
#     ds = ImageFolder(cells_path)
# #     print(ds)
#     indices = torch.randperm(len(ds)).tolist()
# #     print(indices)
#     n_val = math.floor(len(indices) * .15)
# #     print(n_val)
#     train_ds = torch.utils.data.Subset(ds, indices[:-n_val])
# #     print(len(train_ds))
#     val_ds = torch.utils.data.Subset(ds, indices[-n_val:])
# #     print(len(val_ds))

In [16]:
# label2id = {}
# id2label = {}

# for i, class_name in enumerate(ds.classes):
#     label2id[class_name] = str(i)
#     id2label[str(i)] = class_name
  
# print(label2id)
# print(id2label)

{'AKIWNJPXDHCKDF-UHFFFAOYSA-N': '0', 'IAZDPXIOMUYVGZ-UHFFFAOYSA-N': '1', 'IORBSUTUVGBYIQ-UHFFFAOYSA-N': '2', 'LYUZYPKZQDYMEE-UHFFFAOYSA-N': '3', 'NECFYYDWHNCESA-UHFFFAOYSA-N': '4', 'ODYFMVPOLQSSRU-UHFFFAOYSA-N': '5', 'OZXFTSNPYNIBCW-UHFFFAOYSA-N': '6', 'RIFHXYXVPIYGFK-UHFFFAOYSA-N': '7', 'SRVFFFJZQVENJC-UHFFFAOYSA-N': '8', 'WEBULUVFDZSBFP-UHFFFAOYSA-N': '9', 'XNVYXCYPADVFSW-UHFFFAOYSA-N': '10', 'YTLBFPKIYGJKSK-UHFFFAOYSA-N': '11'}
{'0': 'AKIWNJPXDHCKDF-UHFFFAOYSA-N', '1': 'IAZDPXIOMUYVGZ-UHFFFAOYSA-N', '2': 'IORBSUTUVGBYIQ-UHFFFAOYSA-N', '3': 'LYUZYPKZQDYMEE-UHFFFAOYSA-N', '4': 'NECFYYDWHNCESA-UHFFFAOYSA-N', '5': 'ODYFMVPOLQSSRU-UHFFFAOYSA-N', '6': 'OZXFTSNPYNIBCW-UHFFFAOYSA-N', '7': 'RIFHXYXVPIYGFK-UHFFFAOYSA-N', '8': 'SRVFFFJZQVENJC-UHFFFAOYSA-N', '9': 'WEBULUVFDZSBFP-UHFFFAOYSA-N', '10': 'XNVYXCYPADVFSW-UHFFFAOYSA-N', '11': 'YTLBFPKIYGJKSK-UHFFFAOYSA-N'}


In [17]:
# class ImageClassificationCollator:
#     def __init__(self, feature_extractor):
#         self.feature_extractor = feature_extractor
 
#     def __call__(self, batch):
#         encodings = self.feature_extractor([x[0] for x in batch], return_tensors='pt')
#         encodings['labels'] = torch.tensor([x[1] for x in batch], dtype=torch.long)
#         return encodings 

In [18]:
# feature_extractor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')
# model = ViTForImageClassification.from_pretrained(
#     'google/vit-base-patch16-224-in21k',
#     num_labels=len(label2id),
#     label2id=label2id,
#     id2label=id2label
# )
# collator = ImageClassificationCollator(feature_extractor)
# train_loader = DataLoader(train_ds, batch_size=8, collate_fn=collator, num_workers=0, shuffle=True)
# val_loader = DataLoader(val_ds, batch_size=8, collate_fn=collator, num_workers=0)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
# print(len(train_loader))
# print(len(val_loader))

1975
349


In [20]:
# class Classifier(pl.LightningModule):

#     def __init__(self, model, lr: float = 2e-5, **kwargs):
#         super().__init__()
#         self.save_hyperparameters('lr', *list(kwargs))
#         self.model = model
#         self.forward = self.model.forward
#         self.val_acc = Accuracy(
#             task='multiclass' if model.config.num_labels > 2 else 'binary',
#             num_classes=model.config.num_labels
#         )

#     def training_step(self, batch, batch_idx):
#         outputs = self(**batch)
#         self.log(f"train_loss", outputs.loss)
#         return outputs.loss

#     def validation_step(self, batch, batch_idx):
#         outputs = self(**batch)
#         self.log(f"val_loss", outputs.loss)
#         acc = self.val_acc(outputs.logits.argmax(1), batch['labels'])
#         self.log(f"val_acc", acc, prog_bar=True)
#         return outputs.loss

#     def configure_optimizers(self):
#         return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)

In [ ]:
# pl.seed_everything(42)
# classifier = Classifier(model, lr=2e-5)
# trainer = pl.Trainer(accelerator='cpu', devices=1, precision=16, max_epochs=4)
# # trainer = pl.Trainer(accelerator='mps', devices=1, precision=16, max_epochs=4)
# # trainer = pl.Trainer()
# trainer.fit(classifier, train_loader, val_loader)

Seed set to 42
/Users/abestroka/anaconda3/envs/cellprofiler/lib/python3.8/site-packages/lightning_fabric/connector.py:558: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
/Users/abestroka/anaconda3/envs/cellprofiler/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:551: You passed `Trainer(accelerator='cpu', precision='16-mixed')` but AMP with fp16 is not supported on CPU. Using `precision='bf16-mixed'` instead.
Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/abestroka/anaconda3/envs/cellprofiler/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:67: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potent

Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

/Users/abestroka/anaconda3/envs/cellprofiler/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.
[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.
/Users/abestroka/anaconda3/envs/cellprofiler/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.


Training: |                                               | 0/? [00:00<?, ?it/s]